# Introduction

The notebook is intended to perform a Sales Forecast over the data of the [Tabular Playground Series - September 2022](https://www.kaggle.com/competitions/tabular-playground-series-sep-2022)

In [ ]:
# Import Standard Libraries
import pandas as pd

import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv('./../../data/2022_09/train.csv')
test_data = pd.read_csv('./../../data/2022_09/test.csv')

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

# Exploratory Data Analysis

## Row ID

Check if the row id has NaN or duplicated values.

In [ ]:
# Check NaN values
print("Row ID NaN values count: {}".format(len(train_data[train_data['row_id'].isna()])))

In [ ]:
# Check duplicated values
print("Row ID duplicated values count: {}".format(len(test_data[test_data['row_id'].duplicated()])))

## Date

In [ ]:
# Convert the 'date' into a datetime
train_data['date_datetime'] = pd.to_datetime(train_data['date'], format='%Y-%m-%d')
test_data['date_datetime'] = pd.to_datetime(test_data['date'], format='%Y-%m-%d')

In [ ]:
# Extract day, month, year, day of the week and year_month information from 'date'
train_data['date_day'] = train_data['date_datetime'].dt.day
train_data['date_month'] = train_data['date_datetime'].dt.month
train_data['date_year'] = train_data['date_datetime'].dt.year
train_data['date_dayofweek'] = train_data['date_datetime'].dt.dayofweek
train_data['date_year_month'] = pd.to_datetime(train_data['date_datetime']).dt.to_period('M')

test_data['date_day'] = test_data['date_datetime'].dt.day
test_data['date_month'] = test_data['date_datetime'].dt.month
test_data['date_year'] = test_data['date_datetime'].dt.year
test_data['date_dayofweek'] = test_data['date_datetime'].dt.dayofweek
test_data['date_year_month'] = pd.to_datetime(test_data['date_datetime']).dt.to_period('M')

In [ ]:
# Define subplots
figure = make_subplots(rows=2, 
                       cols=2, 
                       subplot_titles=('Date over Years', 
                                       'Date over Months', 
                                       'Date over Days', 
                                       'Date over Weekdays'))


figure.add_trace(go.Scatter(x=train_data.groupby(['date_year']).size().sort_index().index.to_list(),
                            y=train_data.groupby(['date_year']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Yearly'),
                 row=1, 
                 col=1)

figure.add_trace(go.Scatter(x=train_data.groupby(['date_month']).size().sort_index().index.to_list(),
                            y=train_data.groupby(['date_month']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Monthly'), 
                 row=1, 
                 col=2)

figure.add_trace(go.Scatter(x=train_data.groupby(['date_day']).size().sort_index().index.to_list(),
                            y=train_data.groupby(['date_day']).size().sort_index().to_list(), 
                            mode='lines',  
                            name='Daily'), 
                 row=2, 
                 col=1)

figure.add_trace(go.Scatter(x=train_data.groupby(['date_dayofweek']).size().sort_index().index.to_list(),
                            y=train_data.groupby(['date_dayofweek']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Days of Week'), 
                 row=2, 
                 col=2)

figure.update_layout(yaxis_title='Sales', 
                     xaxis=dict(tickvals=train_data.groupby(['date_year']).size().sort_index().index.to_list()),
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.45, 
                     title_text='Train Data Date',
                     height=1000, 
                     template='plotly_dark')

figure.show()

In [ ]:
# Define subplots
figure = make_subplots(rows=2, 
                       cols=2, 
                       subplot_titles=('Date over Years', 
                                       'Date over Months', 
                                       'Date over Days', 
                                       'Date over Weekdays'))


figure.add_trace(go.Scatter(x=test_data.groupby(['date_year']).size().sort_index().index.to_list(),
                            y=test_data.groupby(['date_year']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Yearly'),
                 row=1, 
                 col=1)

figure.add_trace(go.Scatter(x=test_data.groupby(['date_month']).size().sort_index().index.to_list(),
                            y=test_data.groupby(['date_month']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Monthly'), 
                 row=1, 
                 col=2)

figure.add_trace(go.Scatter(x=test_data.groupby(['date_day']).size().sort_index().index.to_list(),
                            y=test_data.groupby(['date_day']).size().sort_index().to_list(), 
                            mode='lines',  
                            name='Daily'), 
                 row=2, 
                 col=1)

figure.add_trace(go.Scatter(x=test_data.groupby(['date_dayofweek']).size().sort_index().index.to_list(),
                            y=test_data.groupby(['date_dayofweek']).size().sort_index().to_list(), 
                            mode='lines', 
                            name='Days of Week'), 
                 row=2, 
                 col=2)

figure.update_layout(yaxis_title='Sales', 
                     xaxis=dict(tickvals=test_data.groupby(['date_year']).size().sort_index().index.to_list()),
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.45, 
                     title_text='Test Data Date',
                     height=1000, 
                     template='plotly_dark')

figure.show()

Conclusions:
- **Years:** The sales are growing from 2019 and this trend is continuing in the test data which interests the 2021
- **Months:** The sales have a similar seasonality between train and test data (a hole in February)
- **Days:** Similar trend between train and test data. A drop during the last days of the month
- **Days of the Week:** <span style="color:red"><b>An inverse seasonality between train and test data.</b></span>

## Country

In [ ]:
# Check NaN values
print("Country NaN values count: {}".format(len(train_data[train_data['country'].isna()])))

In [ ]:
# Plot the Histogram of 'country' Distribution with respect to the Year
figure = ex.histogram(train_data, 
                      x='country', 
                      color='date_year',
                      title='Country per Year Distribution', 
                      labels={'country':'Country',
                              'date_year': 'Year'},
                      barmode='group',
                      height=500,
                      histnorm='',
                      category_orders={'date_year': [2017, 2018, 2019, 2020]},
                      color_discrete_sequence=ex.colors.qualitative.Set3,
                      template='plotly_dark')

figure.update_layout(yaxis_title='Sales', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.45)

# Adjust the x-axis label rotation
figure.update_xaxes(tickangle=45)

figure.show()

Uniform distribution.

## Product

In [ ]:
# Check NaN values
print("Product NaN values count: {}".format(len(train_data[train_data['product'].isna()])))

In [ ]:
figure = ex.pie(train_data, 
                names='product', 
                title='Product', 
                template='plotly_dark',
                color_discrete_sequence=ex.colors.qualitative.Set3,)

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.40)

figure.show()

In [ ]:
# Plot the Histogram of 'product' Distribution with respect to the Country
figure = ex.histogram(train_data, 
                      x='product', 
                      color='country',
                      title='Product per Country Distribution', 
                      labels={'product':'Product',
                              'country': 'Country'},
                      barmode='group',
                      height=500,
                      histnorm='',
                      color_discrete_sequence=ex.colors.qualitative.Set3,
                      template='plotly_dark')

figure.update_layout(yaxis_title='Sales', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.45)

# Adjust the x-axis label rotation
figure.update_xaxes(tickangle=45)

figure.show()

## Number Sold

In [ ]:
# Check NaN values
print("Number Sold NaN values count: {}".format(len(train_data[train_data['num_sold'].isna()])))

In [ ]:
# Plot the Histogram of 'num_sold' Distribution with respect to the Product and the Country
figure = ex.histogram(train_data, 
                      x='country', 
                      y='num_sold',
                      color='product',
                      title='Number of Sold Products per Country Distribution', 
                      labels={'num_sold':'Sales',
                              'product':'Product',
                              'country': 'Country'},
                      barmode='group',
                      height=500,
                      histnorm='',
                      color_discrete_sequence=ex.colors.qualitative.Set3,
                      template='plotly_dark')

figure.update_layout(yaxis_title='Sales', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.45)

# Adjust the x-axis label rotation
figure.update_xaxes(tickangle=45)

figure.show()

The `country`and the `product`, as expected, strongly influence the number of sold pieces.

# Data Preparation

## Product Normalization

In [ ]:
# Replace spaces and special char

In [ ]:
# Define features and labels
